Pytorch models in modAL workflows
=============================

Thanks to Skorch API, you can seamlessly integrate Pytorch models into your modAL workflow. In this tutorial, we shall quickly introduce how to use Skorch API of Keras and we are going to see how to do active learning with it. More details on the Keras scikit-learn API [can be found here](https://skorch.readthedocs.io/en/stable/).

The executable script for this example can be [found here](https://github.com/cosmic-cortex/modAL/blob/master/examples/pytorch_integration.py)!

Skorch API
-----------------------

By default, a Pytorch model's interface differs from what is used for scikit-learn estimators. However, with the use of Skorch wrapper, it is possible to adapt your model.

In [2]:
import torch
from torch import nn
from skorch import NeuralNetClassifier

# build class for the skorch API
class Torch_Model(nn.Module):
    def __init__(self,):
        super(Torch_Model, self).__init__()
        self.convs = nn.Sequential(
                                nn.Conv2d(1,32,3),
                                nn.ReLU(),
                                nn.Conv2d(32,64,3),
                                nn.ReLU(),
                                nn.MaxPool2d(2),
                                nn.Dropout(0.25)
        )
        self.fcs = nn.Sequential(
                                nn.Linear(12*12*64,128),
                                nn.ReLU(),
                                nn.Dropout(0.5),
                                nn.Linear(128,10),
        )

    def forward(self, x):
        out = x
        out = self.convs(out)
        out = out.view(-1,12*12*64)
        out = self.fcs(out)
        return out

For our purposes, the ``classifier`` which we will initialize now acts just like any scikit-learn estimator.

In [3]:
# create the classifier
device = "cuda" if torch.cuda.is_available() else "cpu"
classifier = NeuralNetClassifier(Torch_Model,
                                 criterion=nn.CrossEntropyLoss,
                                 optimizer=torch.optim.Adam,
                                 train_split=None,
                                 verbose=1,
                                 device=device)

Active learning with Pytorch
---------------------------------------

In this example, we are going to use the famous MNIST dataset, which is available as a built-in for PyTorch.

In [4]:
import numpy as np
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision.datasets import MNIST


mnist_data = MNIST('.', download=True, transform=ToTensor())
dataloader = DataLoader(mnist_data, shuffle=True, batch_size=60000)
X, y = next(iter(dataloader))

# read training data
X_train, X_test, y_train, y_test = X[:50000], X[50000:], y[:50000], y[50000:]
X_train = X_train.reshape(50000, 1, 28, 28)
X_test = X_test.reshape(10000, 1, 28, 28)

# assemble initial data
n_initial = 1000
initial_idx = np.random.choice(range(len(X_train)), size=n_initial, replace=False)
X_initial = X_train[initial_idx]
y_initial = y_train[initial_idx]

# generate the pool
# remove the initial data from the training dataset
X_pool = np.delete(X_train, initial_idx, axis=0)[:5000]
y_pool = np.delete(y_train, initial_idx, axis=0)[:5000]

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:13<00:00, 749390.59it/s] 


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 148894.73it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:04<00:00, 402550.56it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 6796478.33it/s]


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



Active learning with data and classifier ready is as easy as always. Because training is *very* expensive in large neural networks, this time we are going to query the best 200 instances each time we measure the uncertainty of the pool.

In [5]:
from modAL.models import ActiveLearner

# initialize ActiveLearner
learner = ActiveLearner(
    estimator=classifier,
    X_training=X_initial, y_training=y_initial,
)

  epoch    train_loss     dur
-------  ------------  ------
      1        2.8759  3.9433
      2        2.3035  0.0360
      3        2.3013  0.0359
      4        2.3004  0.0336
      5        2.2997  0.0348
      6        2.2993  0.0355
      7        2.2986  0.0337
      8        2.2992  0.0357
      9        2.3001  0.0350
     10        2.2994  0.0346


To make sure that you train only on newly queried labels, pass ``only_new=True`` to the ``.teach()`` method of the learner.

In [6]:
# the active learning loop
n_queries = 10
for idx in range(n_queries):
    print('Query no. %d' % (idx + 1))
    query_idx, query_instance = learner.query(X_pool, n_instances=100)
    learner.teach(
        X=X_pool[query_idx], y=y_pool[query_idx], only_new=True,
    )
    # remove queried instance from pool
    X_pool = np.delete(X_pool, query_idx, axis=0)
    y_pool = np.delete(y_pool, query_idx, axis=0)

Query no. 1
Re-initializing module.
Re-initializing optimizer.
  epoch    train_loss     dur
-------  ------------  ------
      1        2.3189  0.0071
      2        4.2473  0.0041
      3        2.6374  0.0041
      4        2.2374  0.0040
      5        2.1513  0.0048
      6        2.0951  0.0044
      7        1.9428  0.0038
      8        1.8126  0.0039
      9        1.5575  0.0049
     10        1.3876  0.0039
Query no. 2
Re-initializing module.
Re-initializing optimizer.
  epoch    train_loss     dur
-------  ------------  ------
      1        2.2986  0.0042
      2       27.0114  0.0046
      3        4.0127  0.0039
      4        2.2890  0.0041
      5        2.3036  0.0041
      6        2.2992  0.0038
      7        2.2942  0.0046
      8        2.2889  0.0041
      9        2.2813  0.0040
     10        2.2751  0.0044
Query no. 3
Re-initializing module.
Re-initializing optimizer.
  epoch    train_loss     dur
-------  ------------  ------
      1        2.3105  0.0042
 